# Storage Formats

**Prerequisites**

- [Intro to DataFrames and Series](https://datascience.quantecon.org/intro.html)  


**Outcomes**

- Understand that data can be saved in various formats  
- Know where to get help on file input and output  
- Know when to use csv, xlsx, feather, and sql formats  


**Data**

- Results for all NFL games between September 1920 to February 2017  

In [ ]:
# Uncomment following line to install on colab
#! pip install 

In [1]:
import pandas as pd
import numpy as np

## File Formats

Data can be saved in a variety of formats.

pandas understands how to write and read DataFrames to and from many of
these formats.

We defer to the [official
documentation](https://pandas.pydata.org/pandas-docs/stable/io.html)
for a full description of how to interact with all the file formats, but
will briefly discuss a few of them here.

### CSV

**What is it?** CSVs store data as plain text (strings) where each row is a
line and columns are separated by `,`.

**Pros**

- Widely used (you should be familiar with it)  
- Plain text file (can open on any computer, “future proof”)  
- Can be read from and written to by most data software  


**Cons**

- Not the most efficient way to store or access  
- No formal standard, so there is room for user interpretation on how to
  handle edge cases (e.g. what to do about a data field that itself includes
  a comma)  


**When to use**:

- A great default option for most use cases  

### xlsx

**What is it?** xlsx is a binary file format used as Excel’s default.

**Pros**:

- Standard format in many industries  
- Easy to share with colleagues that use Excel  


**Cons**:

- Quite slow to read/write large amounts of data  
- Stores both data and *metadata* like styling and display information
  and even plots. This metadata is not always portable to other file formats
  or programs.  


**When to use**:

- When sharing data with Excel  
- When you would like special formatting to be applied to the
  spreadsheet when viewed in Excel  

### Parquet

**What is it?** Parquet is a custom binary format designed for efficient reading and
writing of data stored in columns.

**Pros**:

- *Very* fast  
- Naturally understands all `dtypes` used by pandas, including
  multi-index DataFrames  
- Very common in “big data” systems like Hadoop or Spark  
- Supports various compression algorithms  


**Cons**:

- Binary storage format that is not human-readable  


**When to use**:

- If you have “not small” amounts (> 100 MB) of unchanging data that
  you want to read quickly  
- If you want to store data in an size-and-time-efficient way that may
  be accessed by external systems  

### Feather

**What is it?** Feather is a custom binary format designed for efficient reading and
writing of data stored in columns.

**Pros**:

- *Very* fast – even faster than parquet  
- Naturally understands all `dtypes` used by pandas  


**Cons**:

- Can only read and write from Python and a handful of other
  programming languages  
- New file format (introduced in March ‘16), so most files don’t come
  in this format  
- Only supports standard pandas index, so you need to `reset_index`
  before saving and then `set_index` after loading  


**When to use**:

- Use as an alternative to Parquet if you need the absolute best read and write
  speeds for unchanging datasets  
- Only use when you will not need to access the data in a programming language
  or software outside of Python, R, and Julia  

### SQL

**What is it?** SQL is a language used to interact with relational
databases… [more info](https://en.wikipedia.org/wiki/SQL)

**Pros**:

- Well established industry standard for handling data  
- Much of the world’s data is in a SQL database somewhere  


**Cons**:

- Complicated: to have full control you need to learn another language
  (SQL)  


**When to use**:

- When reading from or writing to existing SQL databases  


**NOTE**: We can cover interacting with SQL databases in a dedicated
lecture – contact us for more information.

## Writing DataFrames

Let’s now talk about saving a DataFrame to a file.

As a general rule of thumb, if we have a DataFrame `df` and we would
like to save to save it as a file of type `FOO`, then we would call
the method named `df.to_FOO(...)`.

We will show you how this can be done and try to highlight some of the
items mentioned above.

But, we will not cover all possible options and features — we feel
it is best to learn these as you need them by consulting the appropriate
documentation.

First, we need some DataFrames to save. Let’s make them now.

Note that by default `df2` will be approximately 10 MB.

If you need to change this number, adjust the value of
the `wanted_mb` variable below.

In [2]:
np.random.seed(42)  # makes sure we get the same random numbers each time
df1 = pd.DataFrame(
    np.random.randint(0, 100, size=(10, 4)),
    columns=["a", "b", "c", "d"]
)

wanted_mb = 10  # CHANGE THIS LINE
nrow = 100000
ncol = int(((wanted_mb * 1024**2) / 8) / nrow)
df2 = pd.DataFrame(
    np.random.rand(nrow, ncol),
    columns=["x{}".format(i) for i in range(ncol)]
)

print("df2.shape = ", df2.shape)
print("df2 is approximately {} MB".format(df2.memory_usage().sum() / (1024**2)))

df2.shape =  (100000, 13)
df2 is approximately 9.918338775634766 MB


### [df.to_csv](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html)

Let’s start with `df.to_csv`.

Without any additional arguments, the `df.to_csv` function will return
a string containing the csv form of the DataFrame:

In [3]:
# notice the plain text format -- one row per line, columns separated by `'`
print(df1.to_csv())

,a,b,c,d
0,51,92,14,71
1,60,20,82,86
2,74,74,87,99
3,23,2,21,52
4,1,87,29,37
5,1,63,59,20
6,32,75,57,21
7,88,48,90,58
8,41,91,59,79
9,14,61,61,46



If we do pass an argument, the first argument will be used as the file name.

In [4]:
df1.to_csv("df1.csv")

Run the cell below to verify that the file was created.

In [5]:
import os
os.path.isfile("df1.csv")

True

Let’s see how long it takes to save `df2` to a file. (Because of the `%%time` at
the top, Jupyter will report the total time to run all code in
the cell)

In [6]:
%%time
df2.to_csv("df2.csv")

CPU times: total: 5.7 s
Wall time: 5.76 s


As we will see below, this isn’t as fastest file format we could choose.

### [df.to_excel](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_excel.html)

When saving a DataFrame to an Excel workbook, we can
choose both the name of the workbook (file) and the name of the sheet
within the file where the DataFrame should be written.

We do this by passing the workbook name as the first argument and the
sheet name as the second argument as follows.

In [7]:
df1.to_excel("df1.xlsx", "df1")

ModuleNotFoundError: No module named 'openpyxl'

pandas also gives us the option to write more than one DataFrame to a
workbook.

To do this, we need to first construct an instance of `pd.ExcelWriter`
and then pass that as the first argument to `df.to_excel`.

Let’s see how this works.

In [ ]:
with pd.ExcelWriter("df1.xlsx") as writer:
    df1.to_excel(writer, "df1")
    (df1 + 10).to_excel(writer, "df1 plus 10")

The

In [ ]:
with ... as ... :

syntax used above is an example of a *context manager*.

We don’t need to understand all the details behind what this means
(google it if you are curious).

For now, just recognize that particular syntax as the way to write
multiple sheets to an Excel workbook.

Saving `df2` to an excel file takes a very long time.

For that reason, we will just show the code and hard-code the output
we saw when we ran the code.

In [ ]:
%%time
df2.to_excel("df2.xlsx")

In [ ]:
 Wall time: 25.7 s

### [pyarrow.feather.write_feather](https://arrow.apache.org/docs/python/generated/pyarrow.feather.write_feather.html#pyarrow.feather.write_feather)

As noted above, the feather file format was developed for very efficient
reading and writing between Python and your computer.

Support for this format is provided by a separate Python package called `pyarrow`.

This package is not installed by default. To install it, copy/paste the code
below into a code cell and execute.

In [8]:
!pip install pyarrow

The parameters for `pyarrow.feather.write_feather` are the DataFrame and file name.

Let’s try it out.

In [9]:
import pyarrow.feather
pyarrow.feather.write_feather(df1, "df1.feather")

In [10]:
%%time
pyarrow.feather.write_feather(df2, "df2.feather")

CPU times: total: 188 ms
Wall time: 44 ms


An example timing result:

|format|time|
|:------------------------------------------------:|:------------------------------------------------:|
|csv|2.66 seconds|
|xlsx|25.7 seconds|
|feather|43 milliseconds|
As you can see, saving this DataFrame in the feather format was far
faster than either CSV or Excel.

## Reading Files into DataFrames

As with the `df.to_FOO` family of methods, there are similar
`pd.read_FOO` functions. (Note: they are in defined pandas, not as
methods on a DataFrame.)

These methods have many more options because data storage can be messy or wrong.

We will explore these in more detail in a separate lecture.

For now, we just want to highlight the differences in how to read data
from each of the file formats.

Let’s start by reading the files we just created to verify that they
match the data we began with.

In [11]:
# notice that index was specified in the first (0th -- why?) column of the file
df1_csv = pd.read_csv("df1.csv", index_col=0)
df1_csv.head()

,a,b,c,d
0,51,92,14,71
1,60,20,82,86
2,74,74,87,99
3,23,2,21,52
4,1,87,29,37


In [ ]:
df1_xlsx = pd.read_excel("df1.xlsx", "df1", index_col=0)
df1_xlsx.head()

In [12]:
# notice feather already knows what the index is
df1_feather = pyarrow.feather.read_feather("df1.feather")
df1_feather.head()

,a,b,c,d
0,51,92,14,71
1,60,20,82,86
2,74,74,87,99
3,23,2,21,52
4,1,87,29,37


With the `pd.read_FOO` family of functions, we can also read files
from places on the internet.

We saved our `df1` DataFrame to a file
and posted it online.

Below, we show an example of using `pd.read_csv` to read this file.

In [13]:
df1_url = "https://raw.githubusercontent.com/QuantEcon/lecture-datascience.myst/main/lectures/pandas/df1.csv"
df1_web = pd.read_csv(df1_url, index_col=0)
df1_web.head()

,a,b,c,d
0,51,92,14,71
1,60,20,82,86
2,74,74,87,99
3,23,2,21,52
4,1,87,29,37


## Practice

Now it’s your turn…

In the cell below, the variable `url` contains a web address to a csv
file containing the result of all NFL games from September 1920 to
February 2017.

Your task is to do the following:

- Use `pd.read_csv` to read this file into a DataFrame named `nfl`  
- Print the shape and column names of `nfl`  
- Save the DataFrame to a file named `nfl.xlsx`  
- Open the spreadsheet using Excel on your computer  


If you finish quickly, do some basic analysis of the data. Try to do
something interesting. If you get stuck, here are some suggestions for
what to try:

- Compute the average total points in each game (note, you will need to
  sum two of the columns to get total points).  
- Repeat the above calculation, but only for playoff games.  
- Compute the average score for your favorite team (you’ll need to
  consider when they were team1 vs team2).  
- Compute the ratio of “upsets” to total games played. An upset is
  defined as a team with a lower ELO winning the game.  

In [15]:
url = "https://raw.githubusercontent.com/fivethirtyeight/nfl-elo-game/"
url = url + "3488b7d0b46c5f6583679bc40fb3a42d729abd39/data/nfl_games.csv"

# your code here --- create more cells if necessary
df = pd.read_csv(url)
df.head()

,date,season,neutral,playoff,team1,team2,elo1,elo2,elo_prob1,score1,score2,result1
0,1920-09-26,1920,0,0,RII,STP,1503.947,1300.000,0.824651,48,0,1.0
1,1920-10-03,1920,0,0,AKR,WHE,1503.420,1300.000,0.824212,43,0,1.0
2,1920-10-03,1920,0,0,RCH,ABU,1503.420,1300.000,0.824212,10,0,1.0
3,1920-10-03,1920,0,0,DAY,COL,1493.002,1504.908,0.575819,14,0,1.0
4,1920-10-03,1920,0,0,RII,MUN,1516.108,1478.004,0.644171,45,0,1.0


In [37]:
# ratio of "upsets" in the data
df.apply(lambda x: (x["elo1"] - x['elo2'] )*(x["score1"] - x['score2'] )<0  , axis=1).mean()

0.34358322744599745

In [41]:
# correct try 1
TEAM_NAME = 'NYG'
df[f'{TEAM_NAME}_score']=df.apply(lambda x: x['score1'] if x['team1']==TEAM_NAME else (x['score2'] if x['team2']==TEAM_NAME else np.nan) , axis=1)
df

,date,season,neutral,playoff,team1,team2,elo1,elo2,elo_prob1,score1,score2,result1,average_score,NYG_score
0,1920-09-26,1920,0,0,RII,STP,1503.947000,1300.000000,0.824651,48,0,1.0,24.0,NaN
1,1920-10-03,1920,0,0,AKR,WHE,1503.420000,1300.000000,0.824212,43,0,1.0,21.5,NaN
2,1920-10-03,1920,0,0,RCH,ABU,1503.420000,1300.000000,0.824212,10,0,1.0,5.0,NaN
3,1920-10-03,1920,0,0,DAY,COL,1493.002000,1504.908000,0.575819,14,0,1.0,7.0,NaN
4,1920-10-03,1920,0,0,RII,MUN,1516.108000,1478.004000,0.644171,45,0,1.0,22.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15735,2017-01-15,2016,0,1,DAL,GB,1617.794683,1635.451172,0.567714,31,34,0.0,32.5,NaN
15736,2017-01-15,2016,0,1,KC,PIT,1681.926463,1647.734179,0.638993,16,18,0.0,17.0,NaN
15737,2017-01-22,2016,0,1,ATL,GB,1664.127266,1651.537731,0.609840,44,21,1.0,32.5,NaN
15738,2017-01-22,2016,0,1,NE,PIT,1747.160321,1662.437215,0.703052,36,17,1.0,26.5,NaN


In [47]:
df.loc[ - df[f'{TEAM_NAME}_score'].isnull()][f'{TEAM_NAME}_score'].mean()

19.20254110612855

In [23]:
# correct try 2
TEAM_NAME = 'NYG'
df.query('team1 == @TEAM_NAME or team2 == @TEAM_NAME').head()

,date,season,neutral,playoff,team1,team2,elo1,elo2,elo_prob1,score1,score2,result1,average_score
422,1925-10-11,1925,0,0,PRV,NYG,1512.773,1300.000,0.831877,14,0,1.0,7.0
428,1925-10-17,1925,0,0,FYJ,NYG,1583.939,1291.915,0.886472,5,3,1.0,4.0
436,1925-10-18,1925,0,0,NYG,FYJ,1289.769,1586.085,0.208901,0,14,0.0,7.0
449,1925-11-01,1925,0,0,NYG,CIB,1279.531,1515.027,0.272603,19,0,1.0,9.5
452,1925-11-03,1925,0,0,NYG,BFF,1326.774,1420.140,0.459269,7,0,1.0,3.5


In [30]:
# wrong try 1
TEAM_NAME = 'NYG'
df_team1_team2_index = df.set_index(['team1','team2'])
df_team1_team2_index.groupby(level=['team1','team2']).filter(lambda x: x['team1']==TEAM_NAME or x['team2']==TEAM_NAME)

KeyError: 'team1'

In [20]:
# wrong try 2
df.loc[df['team1']==TEAM_NAME or df['team2']==TEAM_NAME , : ]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [17]:
df.loc[df['playoff']==1,:]

,date,season,neutral,playoff,team1,team2,elo1,elo2,elo_prob1,score1,score2,result1,average_score
1053,1933-12-17,1933,0,1,CHI,NYG,1665.139000,1630.088000,0.640133,23,21,1.0,22.0
1114,1934-12-09,1934,0,1,NYG,CHI,1581.073000,1765.029000,0.335198,30,13,1.0,21.5
1168,1935-12-15,1935,0,1,DET,NYG,1631.057000,1635.316000,0.586534,26,7,1.0,16.5
1223,1936-12-13,1936,0,1,WSH,GB,1514.744000,1708.815000,0.322350,6,21,0.0,13.5
1279,1937-12-12,1937,0,1,CHI,WSH,1707.518000,1572.545000,0.759719,21,28,0.0,24.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15735,2017-01-15,2016,0,1,DAL,GB,1617.794683,1635.451172,0.567714,31,34,0.0,32.5
15736,2017-01-15,2016,0,1,KC,PIT,1681.926463,1647.734179,0.638993,16,18,0.0,17.0
15737,2017-01-22,2016,0,1,ATL,GB,1664.127266,1651.537731,0.609840,44,21,1.0,32.5
15738,2017-01-22,2016,0,1,NE,PIT,1747.160321,1662.437215,0.703052,36,17,1.0,26.5


In [16]:
df['average_score'] = (df['score1'] + df['score2']) / 2
df.head()

,date,season,neutral,playoff,team1,team2,elo1,elo2,elo_prob1,score1,score2,result1,average_score
0,1920-09-26,1920,0,0,RII,STP,1503.947,1300.000,0.824651,48,0,1.0,24.0
1,1920-10-03,1920,0,0,AKR,WHE,1503.420,1300.000,0.824212,43,0,1.0,21.5
2,1920-10-03,1920,0,0,RCH,ABU,1503.420,1300.000,0.824212,10,0,1.0,5.0
3,1920-10-03,1920,0,0,DAY,COL,1493.002,1504.908,0.575819,14,0,1.0,7.0
4,1920-10-03,1920,0,0,RII,MUN,1516.108,1478.004,0.644171,45,0,1.0,22.5


### Cleanup

If you want to remove the files we just created, run the following cell.

In [14]:
def try_remove(file):
    if os.path.isfile(file):
        os.remove(file)

for df in ["df1", "df2"]:
    for extension in ["csv", "feather", "xlsx"]:
        filename = df + "." + extension
        try_remove(filename)